<a href="https://colab.research.google.com/github/daripp/grape-seed-morphology/blob/main/Pore_Segmentation_and_Volume_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import tifffile as tifffile
import os as os
import numpy as np
import pandas as pd
from skimage import filters, transform, io

from skimage import filters, io, img_as_ubyte
from skimage.data import camera
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import math

from skimage import filters, io
from skimage.data import camera
from skimage import data, io,img_as_float64, img_as_float32, exposure
from skimage.exposure import histogram
from skimage.color import label2rgb, rgb2gray
from scipy import ndimage as ndi
import skimage as sk
# from skimage.external import tifffile
from matplotlib import cm
from matplotlib import pyplot as plt
from skimage import img_as_float64, img_as_int
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import scipy as scipy

import numpy as np

from scipy import ndimage as ndi

from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import remove_small_objects
import porespy as ps
#name your new tiff stack here 
# with tifffile.TiffWriter('/content/drive/MyDrive/FCN and Deeplab for Tiina/amf test.tif', bigtiff=True) as stack:
# #put the file path to the folder of images you want to stack here
#     dir_name = '/content/drive/MyDrive/FCN and Deeplab for Tiina/amf training/masks/'
#     filenames = os.listdir(dir_name)
#     #remove extranious file info from the images here by splitting before and after the image #; this will make sure the images are ordered properly in the stack
#     sort_idx = np.argsort([int(filename.split('1h1')[1].split('.png')[0]) for filename in filenames])
#     for i in sort_idx:
#         filename = dir_name + filenames[i]
        
# #         image=np.array(filename, dtype='ubyte')
# #         image=np.array(filename)
#         stack.save(io.imread(filename))


# %matplotlib inline

#***************************load tiff stack here
data = io.imread('D:/rec20190519_230132_preferm3_cropped_endo_air.tif')
data=np.array(data)
# data=scipy.ndimage.zoom(data,[1,1,1])
data = np.where(data>0, 1, 0)
#data=np.array(data, dtype=int)
#data=img_as_int(data)

print("shape: {}".format(data.shape))
print("dtype: {}".format(data.dtype))
print("range: ({}, {})".format(data.min(), data.max()))
#io.imsave('data.tiff', data)
#io.imshow('data.tiff', cmap='binary')
print(data.shape[1])
print()

#i'm leaving this in because we might be able to use it to alter the spacing of your images.
#The microscope reports the following spacing
# original_spacing = np.array([1, 1, 1])

# # We downsampled each slice 4x to make the data smaller
# rescaled_spacing = original_spacing * [4, 4, 4]

# # Normalize the spacing so that pixels are a distance of 1 apart
# spacing = rescaled_spacing / rescaled_spacing

# print("microscope spacing: {}\n".format(original_spacing))
# print("after rescaling images: {}\n".format(rescaled_spacing))
# print("normalized spacing: {}\n".format(spacing))


def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.set_xticks([])
    ax.set_yticks([])
    
    if title:
        ax.set_title(title)
def slice_in_3D(ax, i):
    # From:
    # https://stackoverflow.com/questions/44881885/python-draw-3d-cube

    import numpy as np
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

    Z = np.array([[0, 0, 0],
                  [1, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0],
                  [0, 0, 1],
                  [1, 0, 1],
                  [1, 1, 1],
                  [0, 1, 1]])

    Z = Z * data.shape

    r = [-1,1]

    X, Y = np.meshgrid(r, r)
    # plot vertices
    ax.scatter3D(Z[:,0], Z[:,1], Z[:,2])

    # list of sides' polygons of figure
    verts = [[Z[0], Z[1], Z[2], Z[3]],
             [Z[4], Z[5], Z[6], Z[7]], 
             [Z[0], Z[1], Z[5], Z[4]], 
             [Z[2], Z[3], Z[7], Z[6]], 
             [Z[1], Z[2], Z[6], Z[5]],
             [Z[4], Z[7], Z[3], Z[0]], 
             [Z[2], Z[3], Z[7], Z[6]]]

    # plot sides
    ax.add_collection3d(
        Poly3DCollection(verts, facecolors=(0, 1, 1, 0.25), linewidths=1,
                         edgecolors='darkblue')
    )

    verts = np.array([[[0, 0, 0],
                       [0, 0, 1],
                       [0, 1, 1],
                       [0, 1, 0]]])
    verts = verts * (data.shape[0], data.shape[1], data.shape[2])
    verts += [i, 0, 0]

    ax.add_collection3d(Poly3DCollection(verts, 
     facecolors='magenta', linewidths=1, edgecolors='black'))

    ax.set_xlabel('plane')
    ax.set_ylabel('col')
    ax.set_zlabel('row')

    # Auto-scale plot axes
    scaling = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    ax.auto_scale_xyz(*[[np.min(scaling), np.max(scaling)]] * 3)

    #plt.show()
    
from ipywidgets import interact

def explore_slices(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data)

    @interact(plane=(0, N - 1))
    def display_slice(plane=(N/2)):
        fig, ax = plt.subplots(figsize=(20, 5))

        ax_3D = fig.add_subplot(133, projection="3d")

        show_plane(ax, data[plane], title=f'Plane {plane}', cmap=cmap)
        slice_in_3D(ax_3D, plane)

        plt.show()

    return display_slice

label_data=label(data)
label_data=morphology.remove_small_objects(label_data,1000)
label_data=label(label_data)
nlabel = len(np.unique(label_data))
relabeled = label2rgb(label_data, bg_label=0, colors=np.random.random((nlabel, 3)))
relabeled2=rgb2gray(relabeled)
explore_slices(relabeled);


#put the new directory you want to save the resliced images to here
directn='E:/'
#put the new folder name here
new_dir_name=directn+'rec20190519_230132_preferm3_cropped_endo_air_Z0_rgb2gray/'
os.makedirs(new_dir_name)
if not os.path.exists(new_dir_name):
      os.makedirs(new_dir_name)
#Put name of filepath and file name for the image stack you want to split here:
# img=io.imread_collection('drive/My Drive/filepath/stack_name.tiff', plugin='tifffile')
img=relabeled2
img=np.array(img)

Z=img.shape
print(Z[0])

#save 
for i in range(Z[0]):
         io.imsave(new_dir_name+'/'+'rec20190519_230132_preferm3_cropped_endo_air%s.png'%i, (img[i,:,:]))

directn='E:/'
#put the new folder name here
new_dir_name=directn+'rec20190519_230132_preferm3_cropped_endo_air__Z0/'
os.makedirs(new_dir_name)
if not os.path.exists(new_dir_name):
      os.makedirs(new_dir_name)
#Put name of filepath and file name for the image stack you want to split here:
# img=io.imread_collection('drive/My Drive/filepath/stack_name.tiff', plugin='tifffile')
img1=relabeled
img1=np.array(img1)

Z=img1.shape
print(Z[0])

#save 
for i in range(Z[0]):
         io.imsave(new_dir_name+'/'+'rec20190519_230132_preferm3_cropped_endo_air%s.png'%i, (img1[i,:,:]))
# relabeled2=np.array(relabeled, dtype='ubyte')
# relabeled2=np.array(relabeled, dtype='ubyte')
# io.imsave('E:/rec20190519_222643_preferm1_cropped_endo_air_relabeled.tiff', relabeled2)
# print("original aka data")


# print("relabeled labels: {}".format(np.unique(relabeled)))
props=regionprops_table(label_data, properties=['label','area'])
df_props=pd.DataFrame(props)





# voxel_size=1.31
surface_areas = ps.metrics.region_surface_areas(regions=label_data)
vols = ps.metrics.region_volumes(regions=label_data,  mode='voxel')


# df_props.insert(4,"Surface Areas", surface_areas)

print(df_props)
fig, ax = plt.subplots(1, 2, figsize=[10, 4])
ax[0].hist(surface_areas, edgecolor='k');
ax[0].set_title("PDF scaling included in the result");
plt.xscale('log');
# surface_areas = ps.metrics.region_surface_areas(regions=label_data)
ax[1].hist(surface_areas, edgecolor='k');
ax[1].set_title("PDF scaling included in the result");
plt.xscale('log')

df_vols=pd.DataFrame(vols, columns=["Volumes"])
df_sa=pd.DataFrame(surface_areas, columns=["Surface Area"])
print(df_vols)
result = pd.concat([df_props, df_vols,df_sa], axis=1)
# result.to_csv('D:/rec20190519_222643_preferm1_cropped_endo_air.csv')
# print(result)

